In [ ]:
# Install Libs
!pip install mediapipe
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task
!pip install mlxtend
!pip install pandas

# Mount Drive.
from google.colab import drive
drive.mount('/content/gdrive')

ERROR: Operation cancelled by user
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

def get_result(image_path):
# Create an FaceLandmarker object.
  base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
  options = vision.FaceLandmarkerOptions(base_options=base_options,
                                        output_face_blendshapes=True,
                                        output_facial_transformation_matrixes=True,
                                        num_faces=1)
  detector = vision.FaceLandmarker.create_from_options(options)
  image = mp.Image.create_from_file(image_path)
  detection_result = detector.detect(image)
  return detection_result

# def get_major_features(result, threshold):
#   features_list = result.face_blendshapes[0]
#   low = min(features_list, key=lambda x: x.score).score
#   high = max(features_list, key=lambda x: x.score).score
#   n = high - low
#   out = [feature.category_name for feature in features_list if (feature.score - low) / n >= threshold]
#   return out

def get_dataset_features(images_list, threshold):
  out = []
  for image, emotion in images_list:
    print(f"{image}")
    result = get_result(image)
    features_list = result.face_blendshapes[0]
    low = min(features_list, key=lambda x: x.score).score
    high = max(features_list, key=lambda x: x.score).score
    n = high - low
    out.append([((feature.score - low) / n) >= threshold for feature in features_list] + [emotion])
  return out

In [ ]:
# import pandas as pd

labels = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprised"]
data = []
for label in labels:
  data.extend(get_dataset_features([(f"/content/gdrive/MyDrive/images/{i}/{label}.jpg", label) for i in range(13)], 0.5))
# for row in data:
#   print(row)

/content/gdrive/MyDrive/images/0/Anger.jpg
/content/gdrive/MyDrive/images/1/Anger.jpg
/content/gdrive/MyDrive/images/2/Anger.jpg
/content/gdrive/MyDrive/images/3/Anger.jpg
/content/gdrive/MyDrive/images/4/Anger.jpg
/content/gdrive/MyDrive/images/5/Anger.jpg
/content/gdrive/MyDrive/images/6/Anger.jpg
/content/gdrive/MyDrive/images/7/Anger.jpg
/content/gdrive/MyDrive/images/8/Anger.jpg
/content/gdrive/MyDrive/images/9/Anger.jpg
/content/gdrive/MyDrive/images/10/Anger.jpg
/content/gdrive/MyDrive/images/11/Anger.jpg
/content/gdrive/MyDrive/images/12/Anger.jpg
/content/gdrive/MyDrive/images/0/Contempt.jpg
/content/gdrive/MyDrive/images/1/Contempt.jpg
/content/gdrive/MyDrive/images/2/Contempt.jpg
/content/gdrive/MyDrive/images/3/Contempt.jpg
/content/gdrive/MyDrive/images/4/Contempt.jpg
/content/gdrive/MyDrive/images/5/Contempt.jpg
/content/gdrive/MyDrive/images/6/Contempt.jpg
/content/gdrive/MyDrive/images/7/Contempt.jpg
/content/gdrive/MyDrive/images/8/Contempt.jpg
/content/gdrive/MyDrive/

In [ ]:
# import math
# def findEntropy(data):
#   t = 0
#   count = {}
#   for label in labels:
#     count[label] = len(data[data["emotion"] == label])
#     t += count[label]
#   entropy = 0
#   try:
#     for label in labels:
#       entropy += -count[label] / t * math.log2(count[label] / t)
#   except:
#     entropy += 0
#   return count, entropy

# def findInformation(data, column):
# 	total = len(data)
# 	information = 0
# 	for attribute in data[column].unique():
# 		mini = data[data[column] == attribute]
# 		count, entropy = findEntropy(mini)
# 		info = (sum(count.values()) / total) * entropy
# 		information += info
# 	return information

# def findGain(data, entropy):
#   return

# def findHighestGain(data):
#   high, label = -float("inf"), ""
#   entropy = findEntropy(data)[1]
#   # print(entropy)
#   gains = {}
#   for column in data.columns[:-1]:
#     # if column == "emotion":
#     #   continue
#     gains[column] = entropy - findInformation(data, column)
#     # print("Before", column, gain, high, label)
#     if gain > high:
#       high = gain
#       label = column
#     # print("After", column, gain, high, label)
#   if label == "":
#     print(data)
#     print(entropy)
#     for column in data.columns:
#       gain = entropy - findInformation(data, column)
#       # print("Before", column, gain, high, label)
#       if gain > high:
#         high = gain
#         label = column
#       # print("After", column, gain, high, label)
#       print(column, gain, high, label)
#   return label

In [ ]:
# class Node:
# 	def __init__(self, attribute):
# 		self.attribute = attribute
# 		self.children = {}

# def getNode(data):
#   label = findHighestGain(data)
#   node = Node(label)
#   print(label)
#   for value in data[label].unique():
#     count, entropy = findEntropy(data[data[label] == value])
#     count_label = ""
#     for k, v in count.items():
#       if v == 0:
#         count_label = k
#         break
#     if entropy == 0:
#       node.children[value] = count_label
#     else:
#       nextData = data[data[label] == value]
#       del nextData[label]
#       node.children[value] = getNode(nextData)
#   return node
# node = getNode(data)

# indent = 0
# def visualise(node, indent):
#   if not node:
#     return
#   print("\t" * indent, end = "")
#   print(node.attribute)
#   for child in node.children.keys():
#     if type(node.children[child]) == str:
#       print("\t" * (indent + 1), end = "")
#       print(f"{child}:{node.children[child]}")
#     else:
#       visualise(node.children[child], indent + 1)

# visualise(node, 0)

     emotion
2      Anger
6      Anger
15  Contempt
29   Disgust
70   Neutral
80       Sad
1.3899750004807707
emotion 1.3899750004807707 1.3899750004807707 emotion
   emotion
5    Anger
83     Sad
0.5
emotion 0.5 0.5 emotion
    emotion
4     Anger
60    Happy
68  Neutral
77  Neutral
85      Sad
0.46438561897747244
emotion 0.46438561897747244 0.46438561897747244 emotion


KeyboardInterrupt: ignored

In [ ]:
# node = getNode(data)
# test = data[5000:]
# correct = 0
# for i in range(5000, 5000 + len(test)):
# 	temp = node
# 	while True:
# 		value = test.loc[i][temp.attribute]
# 		print(temp.attribute, value)
# 		temp = temp.children[value]
# 		if temp == NEG_VALUE or temp == POS_VALUE:
# 			correct += 1
# 			break
# print("Decision Tree (ID3) Algorithm - ")
# print("Train:", len(train))
# print("Test:", len(test))
# print("Correct:", correct)
# print("Effciency:", (correct / len(test)) * 100)

In [ ]:
from collections import defaultdict
import math
class Node:
  def __init__(self):
   self.val = ""
   self.child = {}

def findEntropy(dataset):
  count = defaultdict(int)
  for data in dataset:
    count[data[-1]] += 1
  t = sum(count.values())
  e = 0
  try:
    for key, value in count.items():
      e += -(value / t * math.log2(value / t))
  except:
    e = 0
  return count, e

def findInformation(data):
	vals = defaultdict(lambda: [defaultdict(int), 0])
	for d in data:
			vals[d[0]][0][d[-1]] += 1
	info = 0
	for d in vals.keys():
		count, e = findEntropy(list(filter(lambda x: x[0] == d, data)))
		info += (sum(count.values()) / len(data)) * e
		vals[d][1] = e
	return info, vals

def findGain(data, entropy):
	information, vals = findInformation(data)
	return entropy - information, vals

def findRoot(data, names):
  _, entropy = findEntropy(data)
  gains = {}
  root = Node()
  for i in range(len(data[0]) - 1):
    mini = []
    for d in data:
      mini.append([d[i], d[-1]])
    gain, vals = findGain(mini, entropy)
    gains[names[i]] = [gain, vals]
  best, name = -float("inf"), ""
  for key in gains.keys():
    if gains[key][0] > best:
      best = gains[key][0]
      name = key
  if name == "":
    for k, v in gains.items():
      print(k, "\t\t", v)
    # return
  root.val = name
  vals = gains[name][1]
  for val in vals.keys():
    if vals[val][1] == 0:
      output_label = [key for key in vals[val][0] if vals[val][0][key] == 0]
    else:
      root.child[val] = findRoot(list(filter(lambda x: x[names.index(name)] == val, data)), names)
  return root

headers = ['_neutral', 'browDownLeft', 'browDownRight', 'browInnerUp', 'browOuterUpLeft', 'browOuterUpRight', 'cheekPuff', 'cheekSquintLeft', 'cheekSquintRight', 'eyeBlinkLeft', 'eyeBlinkRight', 'eyeLookDownLeft', 'eyeLookDownRight', 'eyeLookInLeft', 'eyeLookInRight', 'eyeLookOutLeft', 'eyeLookOutRight', 'eyeLookUpLeft', 'eyeLookUpRight', 'eyeSquintLeft', 'eyeSquintRight', 'eyeWideLeft', 'eyeWideRight', 'jawForward', 'jawLeft', 'jawOpen', 'jawRight', 'mouthClose', 'mouthDimpleLeft', 'mouthDimpleRight', 'mouthFrownLeft', 'mouthFrownRight', 'mouthFunnel', 'mouthLeft', 'mouthLowerDownLeft', 'mouthLowerDownRight', 'mouthPressLeft', 'mouthPressRight', 'mouthPucker', 'mouthRight', 'mouthRollLower', 'mouthRollUpper', 'mouthShrugLower', 'mouthShrugUpper', 'mouthSmileLeft', 'mouthSmileRight', 'mouthStretchLeft', 'mouthStretchRight', 'mouthUpperUpLeft', 'mouthUpperUpRight', 'noseSneerLeft', 'noseSneerRight', "emotion"]
root = findRoot(data, headers)

print(root)

NameError: ignored